In [46]:
import os
import csv

def list_testing_files(path, labels, csv_path):
    header = ["Filename", "Emotion", "Emotion_Num"]
    f = open(csv_path, 'w', newline='')
    writer = csv.writer(f)
    writer.writerow(header)
    f.close()

    i = 0
    for label in labels:
        files = os.listdir(os.path.join(path, label))
        for file in files:
            f = open(csv_path, 'a', newline='')
            writer = csv.writer(f)
            writer.writerow([file, label, i])
            f.close()
        i+=1

In [37]:
import shutil

def merge_test_files(path, csv_path):
    with open(csv_path, 'r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        header = csv_reader.fieldnames
        print("start iteration")
        print(header)

        for row in csv_reader:
            try:
                file_name = row['Filename']
                print(file_name)
                file_path = path + row['Emotion'] + '/' + file_name
                new_path = path + file_name

                shutil.move(file_path, new_path)

            except FileNotFoundError:
                print("Error: The specified file does not exist.")
            except Exception as e:
                print(f"Error: {e}")

    print("CSV file reading and iteration completed.")

In [38]:
data_dir = './train_mfcc_label/'

classes = os.listdir(data_dir)
print(classes)

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']


In [47]:
test_dir = './train_mfcc_all/test/'
test_index = './test_index.csv'

list_testing_files(test_dir, classes, test_index)

In [48]:
merge_test_files(test_dir, test_index)

start iteration
['Filename', 'Emotion', 'Emotion_Num']
dia1005_utt20.png
dia1007_utt11.png
dia1012_utt6.png
dia1017_utt3.png
dia1035_utt0.png
dia112_utt3.png
dia138_utt11.png
dia139_utt2.png
dia153_utt11.png
dia173_utt3.png
dia189_utt8.png
dia210_utt4.png
dia23_utt1.png
dia295_utt18.png
dia298_utt5.png
dia300_utt9.png
dia332_utt12.png
dia351_utt5.png
dia393_utt6.png
dia395_utt2.png
dia405_utt3.png
dia430_utt3.png
dia431_utt13.png
dia445_utt14.png
dia456_utt7.png
dia459_utt11.png
dia483_utt1.png
dia493_utt12.png
dia538_utt3.png
dia570_utt0.png
dia572_utt19.png
dia596_utt20.png
dia615_utt22.png
dia626_utt4.png
dia632_utt4.png
dia696_utt7.png
dia6_utt18.png
dia700_utt10.png
dia707_utt0.png
dia717_utt6.png
dia740_utt3.png
dia770_utt14.png
dia772_utt3.png
dia817_utt7.png
dia829_utt2.png
dia839_utt0.png
dia853_utt7.png
dia864_utt4.png
dia884_utt13.png
dia902_utt0.png
dia902_utt15.png
dia908_utt13.png
dia946_utt11.png
dia994_utt6.png
dia996_utt13.png
dia996_utt18.png
dia1010_utt9.png
dia170_u

In [49]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

img_height = 100
img_width = 100

def preprocess_image(path):
    img = load_img(path, target_size = (img_height, img_width))
    a = img_to_array(img)
    a = np.expand_dims(a, axis = 0)
    a /= 255.
    return a

In [53]:
import pandas as pd

test_images_dir = test_dir
test_df = pd.read_csv(test_index)

test_dfToList = test_df['Filename'].tolist()
test_ids = [str(item) for item in test_dfToList]

test_images = [test_images_dir + item for item in test_ids]
test_preprocessed_images = np.vstack([preprocess_image(fn) for fn in test_images])
np.save('./test_preproc_CNN.npy', test_preprocessed_images)

In [58]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model("./CNN_aug_best_weights.h5")

array = model.predict(test_preprocessed_images, batch_size=1, verbose=1)
answer = np.argmax(array, axis=1)

504/504 [==============================] - 6s 12ms/step


In [59]:
test_df = pd.read_csv(test_index)
y_true = test_df['Emotion_Num']
y_pred = array

In [60]:
from sklearn.metrics import log_loss
loss = log_loss(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None)
print(loss)

1.5527074732746515


In [62]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_true, answer)
conf_mat

array([[  0,   0,   0,   0,  56,   0,   0],
       [  0,   0,   0,   0,  14,   0,   0],
       [  0,   0,   0,   0,  14,   0,   0],
       [  0,   0,   0,   0,  88,   0,   0],
       [  0,   0,   0,   0, 236,   0,   0],
       [  0,   0,   0,   0,  35,   0,   0],
       [  0,   0,   0,   0,  61,   0,   0]], dtype=int64)